In [32]:
!pip install --upgrade --quiet  langchain langchain-community langchain-groq neo4j

In [34]:
import os
os.environ["NEO4J_URI"]=NEO4J_URI
os.environ["NEO4J_USERNAME"]=NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"]=NEO4J_PASSWORD

In [35]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)

In [36]:
graph

In [38]:
from langchain_groq import ChatGroq

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7f67ed9d0eb0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f67ed9d3490>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [39]:
from langchain_core.documents import Document
text="""
Elon Reeve Musk (born June 28, 1971) is a businessman and investor known for his key roles in space
company SpaceX and automotive company Tesla, Inc. Other involvements include ownership of X Corp.,
formerly Twitter, and his role in the founding of The Boring Company, xAI, Neuralink and OpenAI.
He is one of the wealthiest people in the world; as of July 2024, Forbes estimates his net worth to be
US$221 billion.Musk was born in Pretoria to Maye and engineer Errol Musk, and briefly attended
the University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through
his Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada.
Musk later transferred to the University of Pennsylvania and received bachelor's degrees in economics
 and physics. He moved to California in 1995 to attend Stanford University, but dropped out after
  two days and, with his brother Kimbal, co-founded online city guide software company Zip2.
 """
documents=[Document(page_content=text)]
documents

[Document(metadata={}, page_content="\nElon Reeve Musk (born June 28, 1971) is a businessman and investor known for his key roles in space\ncompany SpaceX and automotive company Tesla, Inc. Other involvements include ownership of X Corp.,\nformerly Twitter, and his role in the founding of The Boring Company, xAI, Neuralink and OpenAI.\nHe is one of the wealthiest people in the world; as of July 2024, Forbes estimates his net worth to be\nUS$221 billion.Musk was born in Pretoria to Maye and engineer Errol Musk, and briefly attended\nthe University of Pretoria before immigrating to Canada at age 18, acquiring citizenship through\nhis Canadian-born mother. Two years later, he matriculated at Queen's University at Kingston in Canada.\nMusk later transferred to the University of Pennsylvania and received bachelor's degrees in economics\n and physics. He moved to California in 1995 to attend Stanford University, but dropped out after\n  two days and, with his brother Kimbal, co-founded onlin

In [40]:
!pip install --upgrade --quiet langchain_experimental

In [41]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer=LLMGraphTransformer(llm=llm)

In [42]:
graph_documents=llm_transformer.convert_to_graph_documents(documents)

In [43]:
graph_documents

[GraphDocument(nodes=[Node(id='Elon Reeve Musk', type='Person', properties={}), Node(id='Maye', type='Person', properties={}), Node(id='Errol Musk', type='Person', properties={}), Node(id='Kimbal', type='Person', properties={}), Node(id='Spacex', type='Organization', properties={}), Node(id='Tesla, Inc.', type='Organization', properties={}), Node(id='X Corp.', type='Organization', properties={}), Node(id='Twitter', type='Organization', properties={}), Node(id='The Boring Company', type='Organization', properties={}), Node(id='Xai', type='Organization', properties={}), Node(id='Neuralink', type='Organization', properties={}), Node(id='Openai', type='Organization', properties={}), Node(id='University Of Pretoria', type='Educational_institution', properties={}), Node(id="Queen'S University", type='Educational_institution', properties={}), Node(id='University Of Pennsylvania', type='Educational_institution', properties={}), Node(id='Stanford University', type='Educational_institution', pro

In [44]:
graph_documents[0].nodes

[Node(id='Elon Reeve Musk', type='Person', properties={}),
 Node(id='Maye', type='Person', properties={}),
 Node(id='Errol Musk', type='Person', properties={}),
 Node(id='Kimbal', type='Person', properties={}),
 Node(id='Spacex', type='Organization', properties={}),
 Node(id='Tesla, Inc.', type='Organization', properties={}),
 Node(id='X Corp.', type='Organization', properties={}),
 Node(id='Twitter', type='Organization', properties={}),
 Node(id='The Boring Company', type='Organization', properties={}),
 Node(id='Xai', type='Organization', properties={}),
 Node(id='Neuralink', type='Organization', properties={}),
 Node(id='Openai', type='Organization', properties={}),
 Node(id='University Of Pretoria', type='Educational_institution', properties={}),
 Node(id="Queen'S University", type='Educational_institution', properties={}),
 Node(id='University Of Pennsylvania', type='Educational_institution', properties={}),
 Node(id='Stanford University', type='Educational_institution', propertie

In [45]:
graph_documents[0].relationships

[Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Maye', type='Person', properties={}), type='PARENT', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Errol Musk', type='Person', properties={}), type='PARENT', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Kimbal', type='Person', properties={}), type='SIBLING', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Spacex', type='Organization', properties={}), type='FOUNDER', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='Tesla, Inc.', type='Organization', properties={}), type='FOUNDER', properties={}),
 Relationship(source=Node(id='Elon Reeve Musk', type='Person', properties={}), target=Node(id='X Corp.', type='Organization', properties={}), type='

In [46]:
### Load the dataset of movie

movie_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') |
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') |
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') |
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

In [47]:
graph

In [48]:
graph.query(movie_query)

[]

In [49]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Movie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT}
Person {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Movie)-[:IN_GENRE]->(:Genre)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)


In [51]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(llm=llm,graph=graph,verbose=True,allow_dangerous_requests=True )
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x7f67edc0c280>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7f67ed9d0eb0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f67ed9d3490>, model_name='Gemma2-9b-I

In [52]:
response=chain.invoke({"query":"Who was the director of the moview GoldenEye"})

response




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "GoldenEye"})-[r:DIRECTED]->(p:Person) RETURN p.name  

Full Context:
[]

> Finished chain.


{'query': 'Who was the director of the moview GoldenEye',
 'result': "I don't know the answer. \n"}

In [53]:
response=chain.invoke({"query":"tell me the genre of th movie GoldenEye"})

response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "GoldenEye"})<-[:IN_GENRE]-(g:Genre)
RETURN g.name
Full Context:
[]

> Finished chain.


{'query': 'tell me the genre of th movie GoldenEye',
 'result': "I don't know the answer. \n"}

In [54]:
response=chain.invoke({"query":"Who was the director in movie Casino"})

response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "Casino"})<-[:DIRECTED]-(p:Person) RETURN p.name

Full Context:
[{'p.name': 'Martin Scorsese'}]

> Finished chain.


{'query': 'Who was the director in movie Casino',
 'result': 'Martin Scorsese.  \n'}

In [55]:
response=chain.invoke({"query":"Which movie were released in 2008"})

response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie) WHERE m.released = '2008-01-01' RETURN m.title
Full Context:
[]

> Finished chain.


{'query': 'Which movie were released in 2008',
 'result': "I don't know the answer.  \n"}

In [56]:
response=chain.invoke({"query":"Give me the list of movie having imdb rating more than 8"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie) WHERE m.imdbRating > 8 RETURN m.title 

Full Context:
[{'m.title': 'Toy Story'}, {'m.title': 'Heat'}, {'m.title': 'Casino'}, {'m.title': 'Twelve Monkeys (a.k.a. 12 Monkeys)'}, {'m.title': 'Seven (a.k.a. Se7en)'}, {'m.title': 'Usual Suspects, The'}, {'m.title': 'Hate (Haine, La)'}, {'m.title': 'Braveheart'}, {'m.title': 'Taxi Driver'}, {'m.title': 'Anne Frank Remembered'}]

> Finished chain.


{'query': 'Give me the list of movie having imdb rating more than 8',
 'result': "I don't know the answer. \n"}